# Pars 1: URL to transcript, yt hash

In [2]:
from bs4 import BeautifulSoup
import numpy as np

import requests

url = "https://www.juznevesti.com/15-minuta/Marko-Nedeljkovic.sr.html"

req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")

In [3]:
text = ""
for i in soup.find_all("div"):
    current_classes_set = set(i.get("class", list()))
    if current_classes_set == {"bbredbolder", "mb20", "desc_holder"}:
        for j in i.find_all("p"):
            text += j.text + "\n"
video_div = soup.find(id="video_1")
video_link = video_div.contents[1].get("src")
yt_hash = video_link.split("/")[-1]
yt_hash


'Sk4JedgwClY'

In [4]:
video_div.contents

['\n',
 <iframe allowfullscreen="" frameborder="0" height="265" src="https://www.youtube.com/embed/Sk4JedgwClY" width="470"></iframe>,
 '\n']

# Pars 2: generating URLs.

In [19]:
found_urls = []
for pagenumber in range(1, 16):
    
    url = f"https://www.juznevesti.com/15-minuta/index.{pagenumber}.sr.html"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    h3s = soup.find_all("h3")
    h3s_filtered = [i for i in h3s if "title--2" in i.get("class")]
    found_urls.extend(
        [i.a["href"] for i in h3s_filtered]
    )
found_urls = list(set(found_urls))

In [20]:
len(found_urls)

300

# Fusion of the two parts

In [21]:
transcripts = list()
hashes = list()

for url in found_urls:
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    current_text = ""
    for i in soup.find_all("div"):
        current_classes_set = set(i.get("class", list()))
        if "desc_holder" in current_classes_set:
            current_text = "    ".join([j.text + "\n" for j in i.find_all("p")])
    # video_div = soup.find(id="video_1")
    # for element in video_div.contents:
    #     try:
    #         video_link = element.get("src", np.nan)
    #         break
    #     except:
    #         continue
    iframes = soup.find_all("iframe")

    video_link = [i.get("src") for i in iframes  if "youtube.com" in i.get("src")][0]
    yt_hash = video_link.split("/")[-1]
    transcripts.append(current_text)
    hashes.append(yt_hash)

In [22]:
list(map(len, [hashes, transcripts, found_urls]))

[300, 300, 300]

In [23]:

import pandas as pd
df = pd.DataFrame(data={

    "url": found_urls,
    "yt_hash": hashes,
    "transcript": transcripts,
})

df

,url,yt_hash,transcript
0,https://www.juznevesti.com/15-minuta/Boban-Sto...,00DtyZ2sCUM,"Gost: Boban Stojanović, redovni profesor Ekono..."
1,https://www.juznevesti.com/15-minuta/Slavoljub...,58xZSVbpgkk,"Gost: Slavoljub Vlajković, narodni poslanik SP..."
2,https://www.juznevesti.com/15-minuta/Zoran-Ziv...,YKAqDCyINz4,"Gost: Zoran Žuvković, predsednik Nove stranke\..."
3,https://www.juznevesti.com/15-minuta/Majkl-Dev...,2QShGnPuVjQ,"Gost: Majkl Devenport, Šef Delegacije EU u Srb..."
4,https://www.juznevesti.com/15-minuta/Jelena-Dj...,khEIELaXKwI,"Gost: Jelena Đorđević, Odbornica u Skupštini g..."
...,...,...,...
295,https://www.juznevesti.com/15-minuta/Dragan-Pe...,a1cN7-Y29_g,"Gost: Dragan Petković, savetnik za evropske in..."
296,https://www.juznevesti.com/15-minuta/Vladica-D...,6CTWDc4amzU,"Gost: Vladica Dimitrov, narodni poslanik SNS i..."
297,https://www.juznevesti.com/15-minuta/Spasoje-M...,IUIKV9XlWEM,"Gost: Spasoje Milovanović, direktor Narodnog p..."
298,https://www.juznevesti.com/15-minuta/Vukosava-...,2-F5yoHSn5k,Gosti: Vukosava Crnjanski i Jovana Đurbabić\n ...


In [24]:
df.isna().sum(axis=0)

url           0
yt_hash       0
transcript    0
dtype: int64

In [25]:
df.to_csv("006_crawling_juznevesti.csv", index=False)